In [47]:
import pandas as pd

In [48]:
#from MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter

In [49]:
from surprise import Dataset
from surprise import Reader

In [50]:
df = pd.read_csv("C:\\Users\\Ankit\\Documents\\RecSys-Materials\\ml-latest-small\\collabrative-csv.csv") 


In [51]:
df

,userId,movieId,rating,timestamp
0,SK4001,6,1,NaN
1,SK4001,44,1,NaN
2,SK4001,46,1,NaN
3,SK4001,97,1,NaN
4,SK4001,140,1,NaN
...,...,...,...,...
204840,SK19245,1376,1,NaN
204841,SK19245,1377,1,NaN
204842,SK19245,76251,1,NaN
204843,SK19245,76277,1,NaN


In [52]:
df1 = df.drop(columns=['timestamp'])

In [53]:
df1

,userId,movieId,rating
0,SK4001,6,1
1,SK4001,44,1
2,SK4001,46,1
3,SK4001,97,1
4,SK4001,140,1
...,...,...,...
204840,SK19245,1376,1
204841,SK19245,1377,1
204842,SK19245,76251,1
204843,SK19245,76277,1


In [54]:
df1.rename(columns = {'userId':'userID', 'movieId':'itemID'}, inplace = True) 

In [55]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df1[['userID', 'itemID', 'rating']], reader)

In [57]:
testSubject = 'SK4004'

k = 10

trainSet = data.build_full_trainset()

sim_options = {'name': 'cosine',
               'user_based': True
               }

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

surprise.dump.dump(file_name, predictions=None, algo=None, verbose=0)



# Get top N similar users to our test subject
# (Alternate approach would be to select users up to some similarity threshold - try it!)
testUserInnerID = trainSet.to_inner_uid(testSubject)
similarityRow = simsMatrix[testUserInnerID]

similarUsers = []
for innerID, score in enumerate(similarityRow):
    if (innerID != testUserInnerID):
        similarUsers.append( (innerID, score) )

kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])

# Get the stuff they rated, and add up ratings for each item, weighted by user similarity
candidates = defaultdict(float)
for similarUser in kNeighbors:
    innerID = similarUser[0]
    userSimilarityScore = similarUser[1]
    theirRatings = trainSet.ur[innerID]
    for rating in theirRatings:
        candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore
    
# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1
    
# Get top-rated items from similar users:
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        # movieID = trainSet.to_raw_iid(itemID)
        # print(ml.getMovieName(int(movieID)), ratingSum)
        print(itemID)
        pos += 1
        if (pos > 10):
            break

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
38
85
86
29
111
150
27
42
148
195
270
